# The Battle of Neighborhoods
## Toronto neighborhood - Vegan restaurants

**Author**: Hércules Sant'Ana da Silva José

_Report created in July/2020_

## Summary

**Introduction Section:**
- Introduction where we discuss the business problem about vegan restaurants in Toronto and who would be interested in this project.

**Data Section:**
- Data where we describe the data that will be used to solve the problem and the source of the data.

**Methodology section:**
- Methodology section which represents the main component of this report where we discuss and describe any exploratory data analysis that we did, any inferential statistical testing that we performed, if any, and what machine learnings were used and why.

**Results section:**
- Results section where we discuss the results.

**Discussion section:**
- Discussion section where we discuss any observations you noted and any recommendations to our audience based on the results.

**Conclusion section:**
- Conclusion section where we conclude the report.

## 1. Introduction

### 1.1. Scenario

Recent studies suggest that about 8% of the world's population is vegan. Few people around the world are committed to living a plant-based lifestyle. If you are interested in eating more cleanly, do not want to explore animals or simply enjoy how you feel as a vegan, it can sometimes be difficult to find great vegan restaurants in the world.

However, this is starting to change. You will find that the main cities in the world are slowly but surely developing strong vegan cultures, from coffee shops and cooperatives to some of the best vegan restaurants in the world. In short, you now have more options than simply ordering the only salad on the menu.

Toronto is one of the most ethnic and culturally diverse cities in the world. Located on the shores of Lake Ontario, it has people from all over the world from different cultures and ethnicities resulting in a lot of delicious delights. No matter what type of cuisine you like, you will surely find great restaurants that serve delicious food for you. Now, if you're a vegan, you may be wondering, all of this is good, but what if I'm looking for wonderful vegan cuisine to enjoy in Toronto? Will I be able to find something good? Will I as a student find good vegan restaurants near universities? Which neighborhoods can a company that specializes in vegan products promote its products? 

### 1.2. Business problem

Identify existing vegan restaurants in Toronto close to the city's universities, and identify neighborhoods where vegan companies can partner with restaurants to promote their products.

### 1.3. Audience

Vegan audience that comes to Toronto to study and companies interested in selling and promoting vegan products.

## 2. Data

### 2.1. Data description

For this analysis, we will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, the neighborhood data is not readily available on the internet. So, we will use a Wikipedia page that contains all the information we need to explore and cluster the neighborhoods in Toronto. The wikipedia page is available in https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

We will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format that we can do our analysis.

We will use the Foursquare API to identify the existing vegan restaurants in Toronto and existing restaurants near universities with a distance up to 3000 meters. We will use a list of boroughs and neighborhoods of Toronto with their coordinates (latitude and longitude) to obtain the venues of each boroughs and neighborhoods of Toronto. The coordinates dataset is available in http://cocl.us/Geospatial_data.

### 2.2. Data cleaning

The data cleaning process consisted of obtaining postal codes for the city of Toronto from Wikipedia page. All borough and neighborhood with value "Not assigned" were removed from the dataframe.

In [ ]:
import pandas as pd

# Create dataframe by using pandas and show first 10 lines
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df[0].head(10)

In [ ]:
# We've assigned index 0 to a new Pandas dataframe
df_toronto = df[0]

# Drop cells with a borough that is Not assigned
index_NA = df_toronto[(df_toronto['Borough']=='Not assigned')].index
df_toronto.drop(index_NA, inplace=True)
df_toronto.head(10)

In [ ]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto.loc[df_toronto.Neighbourhood=='Not assigned','Neighbourhood'] = df_toronto.Neighbourhood
df_toronto.head(10)

In [ ]:
df_toronto.shape

Then we obtain the geographic data of the Toronto neighborhoods. We merged the two dataframe, creating a new one with all neighborhoods and their respective coordinates (latitude and longitude).

In [ ]:
# We used Pandas to read csv file that has the geographical coordinates of each postal code to a dataframe
df_coordinators = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinators.head(10)

In [ ]:
# We merged the df_coordinators into df_toronto.
df_toronto = pd.merge(left=df_toronto,right=df_coordinators,on='Postal Code')
df_toronto.head(10)

In [ ]:
# Check numbers of Boroughs and Neighborhoods in the df_toronto
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

We verify the geographic data collected by plotting the neighborhoods on the map.

In [ ]:
import folium

# Toronto, Ontario
latitude = 43.6529
longitude = -79.3849

#Create map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

#add markers to map for neighborhoods
for lat,lng, borough, neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

### 2.3. Checking Foursquare API data

We invoke the Foursquare API to check the returned data and verify how many venues exists around the University of Toronto at a distance of up to 1000 meters.

In [ ]:
# University of Toronto, Toronto, Ontario
neighborhood_latitude = 43.72149
neighborhood_longitude = -79.37881

In [ ]:
# Foursquare credencials
CLIENT_ID = 'ERPDIEJOTYQ15D5OEZPG2BNWRX2XANXE5FS3A5XYM5ESFC4L' # your Foursquare ID
CLIENT_SECRET = 'VTEOE3I0I5SPVKSYMAURWWRCPBVR1GWBADW4H5KOX4JU4044' # your Foursquare Secret
VERSION = '20180604'

# URL
LIMIT=100
radius=1000
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT
)

In [ ]:
import requests # library to handle requests

results = requests.get(url).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

I've cleaned the returned json and I've structured it into a pandas dataframe.

In [ ]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# First 10 rows
nearby_venues.head(10)

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

## 3. Methodology

Using the Foursquare API, we will pick up all the vegan restaurants in each neighborhood in the city of Toronto. In addition, we will have all locations that contain 'university' in their name.

### 3.1. Finding the vegan restaurants and universities

We created a function that returns all venues in the Foursquare API with their respective geographic data. Each venus is associated with its respective neighborhood.

In [ ]:
# Function that return the nearby venues
# Distance: 500 meters
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                   'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
df_toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'],
                                  )

In [ ]:
df_toronto_venues.head(10)

In [ ]:
# Saving the results
df_toronto_venues.to_csv('toronto_venues.csv', index=False)

For performance reasons, the data collected from Foursquare API was saved in a CSV file that it can be read later.

After that, we store on a dataframe only the venues containing 'vegan' or 'university'.

In [107]:
# Loading the saved results
df_toronto_venues = pd.read_csv('toronto_venues.csv')
df_toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


In [108]:
print('Unique categories: {}.'.format(len(df_toronto_venues['Venue Category'].unique())))

Unique categories: 269.


In [109]:
# Verify that it exists a venue or venue category for 'vegan'
for i in df_toronto_venues['Venue Category'].unique():
    if 'Vegan' in i:
        print('Venue Category: ', True)
        break

for i in df_toronto_venues['Venue Category'].unique():
    if 'Vegan' in i:
        print('Venue: ', True)
        break

Venue Category:  True
Venue:  True


In [110]:
# Verify that it exists a venue or venue category for 'unversity'
for i in df_toronto_venues['Venue Category'].unique():
    if 'University' in i:
        print('Venue Category: ', True)
        break

for i in df_toronto_venues['Venue'].unique():
    if 'University' in i:
        print('Venue: ', True)
        break

Venue:  True


## 4. Results

## 5. Discussion

## 6. Conclusion

## 7. References

https://bigseventravel.com/2020/02/best-vegan-restaurants-in-the-world-2020/

https://wtvox.com/sustainable-living/2019-the-world-of-vegan-but-how-many-vegans-are-in-the-world/

https://www.dailyhawker.ca/vegan-restaurants-in-toronto/

https://coursera.org
